In [1]:
import sys
sys.path.append('../')

In [2]:
from textpair.preprocess.dummy_preprocessor import DummyPreprocessor
from textpair.analyze.bert_analyzer import BertAnalyzer
from textpair.vectorize.bert_vectorizer import BertVectorizer
from textpair.model.bert_model import BertModel
from textpair.pair_ann import PairAnn
from textpair.semantic.base_semantic import BaseSemantic

import logging

logging.basicConfig(level=logging.INFO)

import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [3]:
BERT_MODEL_FILE = '/home/aaron/model_files/bert/pytorch/bert-base-chinese.tar.gz'
BERT_VOCAB_FILE = '/home/aaron/model_files/bert/pytorch/bert-base-chinese-vocab.txt'

In [4]:
model = BertModel.from_pretrained(BERT_MODEL_FILE)
tokenizer = BertTokenizer.from_pretrained(BERT_VOCAB_FILE)

INFO:pytorch_pretrained_bert.modeling:loading archive file /home/aaron/model_files/bert/pytorch/bert-base-chinese.tar.gz
INFO:pytorch_pretrained_bert.modeling:extracting archive file /home/aaron/model_files/bert/pytorch/bert-base-chinese.tar.gz to temp dir /tmp/tmpcxayey2y
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "type_vocab_size": 2,
  "vocab_size": 21128
}

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file /home/aaron/model_files/bert/pytorch/bert-base-chinese-vocab.txt


In [5]:
text = "[CLS]明天天气怎么样 ? [SEP] 明天天气如何？ [SEP]"
tokenized_text = tokenizer.tokenize(text)
tokenized_text

['[CLS]',
 '明',
 '天',
 '天',
 '气',
 '怎',
 '么',
 '样',
 '?',
 '[SEP]',
 '明',
 '天',
 '天',
 '气',
 '如',
 '何',
 '？',
 '[SEP]']

In [6]:
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
indexed_tokens

[101,
 3209,
 1921,
 1921,
 3698,
 2582,
 720,
 3416,
 136,
 102,
 3209,
 1921,
 1921,
 3698,
 1963,
 862,
 8043,
 102]

In [7]:
segments_ids = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]

In [8]:
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

In [9]:
with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, segments_tensors)

In [10]:
encoded_layers[-1].numpy().squeeze().shape

(18, 768)

In [11]:
import numpy as np
tmp = np.array([1,2,3])